In [1]:
from spleeter.separator import Separator
import os
import time
import threading
import sys
import subprocess



2024-08-16 20:48:17.589719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-16 20:48:17.589736: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def get_file_size(file_path):
    return os.path.getsize(file_path)

def estimate_progress(start_time, file_size, processed_size):
    elapsed_time = time.time() - start_time
    if processed_size == 0:
        return 0, "Unknown"
    progress = (processed_size / file_size) * 100
    estimated_total_time = (elapsed_time / progress) * 100
    remaining_time = estimated_total_time - elapsed_time
    return progress, time.strftime("%M:%S", time.gmtime(remaining_time))





In [3]:
def animate_progress(stop, file_path):
    file_size = get_file_size(file_path)
    start_time = time.time()
    while not stop():
        try:
            processed_size = sum(get_file_size(os.path.join(root, name))
                                 for root, _, files in os.walk(os.path.dirname(file_path))
                                 for name in files if name.endswith('.wav'))
        except:
            processed_size = 0

        progress, remaining = estimate_progress(start_time, file_size, processed_size)
        sys.stdout.write(f"\rProgress: {progress:.1f}% | Est. remaining time: {remaining}     ")
        sys.stdout.flush()
        time.sleep(1)


In [4]:


def convert_audio(input_file, output_file, format='mp3', quality='128k'):
  # Use ffmpeg to convert and set quality
  command = [
    'ffmpeg', '-i', input_file,
    '-ab', quality,
    output_file
  ]
  subprocess.run(command, check=True)

def separate_instruments(input_dir, output_dir, output_format='mp3', quality='128k', num_files=None):
  separator = Separator('spleeter:5stems')  
  os.makedirs(output_dir, exist_ok=True)
  audio_files = [f for f in os.listdir(input_dir) if f.endswith(('.mp3', '.wav', '.flac', '.ogg'))]

 
  if num_files:
    audio_files = audio_files[:num_files]

  total_files = len(audio_files)

  for index, audio_file in enumerate(audio_files, 1):
    input_path = os.path.join(input_dir, audio_file)
    base_name = os.path.splitext(audio_file)[0]
    file_output_dir = os.path.join(output_dir, base_name)

    print(f"\nProcessing file {index} of {total_files}: {audio_file}")
    print("Separating: ")

    stop_animation = False
    animation_thread = threading.Thread(target=animate_progress, args=(lambda: stop_animation, input_path))
    animation_thread.start()

    start_time = time.time()
    separator.separate_to_file(input_path, file_output_dir)
    end_time = time.time()

    stop_animation = True
    animation_thread.join()

    # Rename the output files and convert format/quality
    for root, _, files in os.walk(file_output_dir):
      for file in files:
        if file.endswith(('.mp3', '.wav')):
          stem_name = file.split('_')[0]  
          new_name = f"{stem_name}_{base_name}.{output_format}"
          input_file_path = os.path.join(root, file)
          output_file_path = os.path.join(root, new_name)

          # Convert audio file
          convert_audio(input_file_path, output_file_path, format=output_format, quality=quality)
          # Delete the original WAV file  (not input one)
          os.remove(input_file_path)

    duration = end_time - start_time
    print(f"\nSeparation complete for {audio_file}.")
    print(f"Time taken: {duration:.2f} seconds")
    print(f"Output saved in {file_output_dir}")
    print(f"Progress: {index}/{total_files} files processed")

  print(f"\nAll separations complete. Total files processed: {total_files}")
  print(f"Output files saved in {output_dir}")


In [ ]:
# if the code below gives some eager error :
# import tensorflow as tf
# tf.compat.v1.disable_eager_execution()
# this error is caused due to racing in multithreading

In [5]:

input_dir = "data/data"
output_dir = "seperated"

separate_instruments(input_dir, output_dir,num_files=10)    


Processing file 1 of 10: 075395.mp3
Separating: 
Progress: 0.0% | Est. remaining time: Unknown     INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models/5stems', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use output_signature instead
Inst

2024-08-16 20:48:19.242546: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-08-16 20:48:19.242569: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: desktop
2024-08-16 20:48:19.242574: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: desktop
2024-08-16 20:48:19.242640: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 550.107.2
2024-08-16 20:48:19.242656: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 550.107.2
2024-08-16 20:48:19.242660: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 550.107.2


Progress: 0.0% | Est. remaining time: Unknown     INFO:tensorflow:Apply unet for piano_spectrogram
INFO:tensorflow:Apply unet for drums_spectrogram
Progress: 0.0% | Est. remaining time: Unknown     INFO:tensorflow:Apply unet for bass_spectrogram
INFO:tensorflow:Apply unet for other_spectrogram
Progress: 0.0% | Est. remaining time: Unknown     INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from pretrained_models/5stems/model


2024-08-16 20:48:22.647396: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-16 20:48:22.704359: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Progress: 0.0% | Est. remaining time: Unknown     INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Progress: 0.0% | Est. remaining time: Unknown     INFO:spleeter:File seperated/075395/075395/vocals.wav written succesfully
INFO:spleeter:File seperated/075395/075395/piano.wav written succesfully
INFO:spleeter:File seperated/075395/075395/drums.wav written succesfully
INFO:spleeter:File seperated/075395/075395/bass.wav written succesfully
INFO:spleeter:File seperated/075395/075395/other.wav written succesfully


...

[out#0/mp3 @ 0x5899730566c0] video:0KiB audio:469KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.095995%
size=     469KiB time=00:00:29.97 bitrate= 128.3kbits/s speed= 115x    


Progress: 0.0% | Est. remaining time: Unknown     INFO:spleeter:File seperated/136928/136928/vocals.wav written succesfully
INFO:spleeter:File seperated/136928/136928/piano.wav written succesfully
INFO:spleeter:File seperated/136928/136928/bass.wav written succesfully
INFO:spleeter:File seperated/136928/136928/drums.wav written succesfully
INFO:spleeter:File seperated/136928/136928/other.wav written succesfully


ffmpeg version n7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.1.1 (GCC) 20240522
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-frei0r --enable-gmp --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libdvdnav --enable-libdvdread --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libharfbuzz --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libplacebo --enable-libpulse --enable-librav1e --enable-librsvg --enable-librubberband --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enab


Separation complete for 136928.mp3.
Time taken: 1.73 seconds
Output saved in seperated/136928
Progress: 10/10 files processed

All separations complete. Total files processed: 10
Output files saved in seperated


[out#0/mp3 @ 0x602fad1fb6c0] video:0KiB audio:469KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.095995%
size=     469KiB time=00:00:29.97 bitrate= 128.3kbits/s speed= 105x    
